# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/3fwVL/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree()
            

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120536e+02     1.362940e+00
 * time: 0.6233298778533936
     1     1.122088e+01     9.934351e-01
 * time: 1.7267489433288574
     2    -1.213786e+01     9.686975e-01
 * time: 2.2224409580230713
     3    -3.385689e+01     7.979538e-01
 * time: 2.985687017440796
     4    -4.742984e+01     6.528826e-01
 * time: 3.7090070247650146
     5    -5.684245e+01     2.181597e-01
 * time: 4.443939924240112
     6    -5.973426e+01     1.518820e-01
 * time: 4.942529916763306
     7    -6.083742e+01     5.135099e-02
 * time: 5.446016073226929
     8    -6.126202e+01     6.828077e-02
 * time: 5.930806875228882
     9    -6.154528e+01     4.022129e-02
 * time: 6.417121887207031
    10    -6.178213e+01     2.797883e-02
 * time: 6.904478073120117
    11    -6.196934e+01     2.112961e-02
 * time: 7.405834913253784
    12    -6.204517e+01     1.492117e-02
 * time: 7.89101505279541
    13    -6.211826e+01     1.400862e-02
 * time: 8.378428936004639
    

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671072
    AtomicLocal         -18.8557721
    AtomicNonlocal      14.8522676
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485384 
    Xc                  -19.3336825

    total               -62.261666462293